In [107]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame
from risk_metrics import Risk_Metrics

In [108]:
connection = create_connection("../database/crypto_billionairs.db")

In [109]:
def momentum(df, lag):
    return df.pct_change(periods=lag)

In [110]:
class Momentum_AR_raw_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.init_long_signal = self.I(init_buy_signal, self.data.df)
        self.init_close_long_signal = self.I(init_close_long_signal, self.data.df)
        self.init_short_signal = self.I(init_short_signal, self.data.df)
        self.init_close_short_signal = self.I(init_close_short_signal, self.data.df)

        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.init_long_signal[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.init_close_long_signal[-1] == 1 and self.position.is_long:
             self.position.close()
             
        elif self.init_short_signal[-1] == -1 and self.position.is_short:
            self.position.close()
            self.sell()
            
        elif self.init_close_short_signal[-1] == -1 and self.position.is_short:
            self.position.close()
            

In [111]:
def init_buy_signal(trash):
    return df_temp['buy_indicator'].shift(1)

def init_short_signal(trash):
    return df_temp['short_indicator'].shift(1)

def init_close_long_signal(df):
    return df_temp['close_buy_indicator'].shift(1)

def init_close_short_signal(df):
    return df_temp['close_short_indicator'].shift(1)

In [112]:
def run_backtesting_raw(db_connection):
    
    global df_temp
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and not 'equity_curve' in name and not "_pooling" in name and "_ensemble" not in name]
    print(filtered_table_names)
    df_risk = pd.DataFrame(columns = range(11))
    df_risk.columns = ["table_name", "timehorizon", "return", "buy and hold return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        df_temp = df_temp[-365:]
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        #trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        #equity_curve = pd.DataFrame(stats["_equity_curve"])
        #equity_curve.to_sql(f"{table}_equity_curve", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp, 0, stats)
        risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], stats["Buy & Hold Return [%]"], stats["Return [%]"],
                             risk.sharpe_ratio("return"), risk.sortino_ratio("return"),
                             risk.max_drawdown(), risk.calmar_ratio(), len(trades), stats["Win Rate [%]"]]
        print(risk_metrics_list)
        #df_risk.append(risk_metrics_list)
        df_risk.loc[len(df_risk)] = risk_metrics_list
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_abnormal_day", db_connection, if_exists="replace")

In [113]:
run_backtesting_raw(connection)

['ADA_1min_complete_1day_preprocessed_1day_features', 'BAT_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BNT_1min_complete_1day_preprocessed_1day_features', 'BSV_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'BTG_1min_complete_1day_preprocessed_1day_features', 'DASH_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'EOS_1min_complete_1day_preprocessed_1day_features', 'ETC_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'FUN_1min_complete_1day_preprocessed_1day_features', 'ICX_1min_complete_1day_preprocessed_1day_features', 'KNC_1min_complete_1day_preprocessed_1day_features', 'LINK_1min_complete_1day_preprocessed_1day_features', 'LRC_1min_complete_1day_preprocessed_1day_features', 'LSK_1min_complete_1day_preprocessed_1day_features', 'LTC_1min_complete_1day_preprocessed_1day_

<ipython-input-112-e930bdb44e79>:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)



['BNT_1min_complete_1day_preprocessed_1day_features', 365, 64.77305002450044, -64.90461253663804, 64.77305002450044, 9.662544419857705, 1.389047032847732, 0.26206031598489116, 2.4716848020680424, 38, 63.1578947368421]
['BSV_1min_complete_1day_preprocessed_1day_features', 365, 76.86894167770055, -61.065740289289785, 76.86894167770055, 9.702261593565698, 1.5692386983798152, 0.16872348345138719, 4.555912437633385, 40, 52.5]
['BTC_1min_complete_1day_preprocessed_1day_features', 365, 12.802699726480379, -21.536649743714253, 12.802699726480379, 2.9322257461349115, 0.26036409451327847, 0.26460079065690284, 0.48384963985542745, 44, 45.45454545454545]
['BTG_1min_complete_1day_preprocessed_1day_features', 365, 7.982171997100347, 1.760497957266667, 7.982171997100347, 1.1120628843794738, 0.16590659554322115, 0.2683687102727155, 0.29743303490890893, 37, 51.35135135135135]
['DASH_1min_complete_1day_preprocessed_1day_features', 365, 124.16226659860061, -47.38420334861406, 124.16226659860061, 15.0861

c:\Users\janfa\anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [114]:
class Momentum_AR_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.init_long_short_signal = self.I(init_long_short_signal, self.data.df)
        self.init_close_long_short_signal = self.I(init_close_long_short_signal, self.data.df)
       

        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if int(float(self.init_long_short_signal[-1])) == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif int(float(self.init_close_long_short_signal[-1])) == 1 and self.position.is_long:
             self.position.close()
            
        elif int(float(self.init_long_short_signal[-1])) == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()
            
        elif int(float(self.init_close_long_short_signal[-1])) == -1 and self.position.is_short:
            self.position.close()

In [115]:
def init_long_short_signal(some_input):
    return df_temp_ml['buy_short_indicator']

def init_close_long_short_signal(some_input):
    return df_temp_ml['close_buy_short_indicator']

In [119]:
def run_backtesting_ml(db_connection):
    
    global df_temp_ml
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if ("_ensemble" in name or "_svc" in name or "_knn" in name or "_random_forest" in name or "_logistic_regression" in name or "_mlp" in name) and 'trades' not in name and not 'equity_curve' in name]
    df_risk = pd.DataFrame(columns = range(11))
    df_risk.columns = ["table_name", "timehorizon", "return", "buy and hold return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp_ml = pd.read_sql_query(f"select * from {table}", db_connection)
        df_temp_ml["return"] = momentum(df_temp_ml["close"], 1) + 1
        
        #df_temp_ml = df_temp_ml[-356:]
        
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp_ml[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_AR_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        #trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        #equity_curve = pd.DataFrame(stats["_equity_curve"])
        #equity_curve.to_sql(f"{table}_equity_curve", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp_ml, 0, stats)
        
        try:
            risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], stats["Buy & Hold Return [%]"], stats["Return [%]"],
                             risk.sharpe_ratio("return"), risk.sortino_ratio("return"),
                             risk.max_drawdown(), risk.calmar_ratio(), len(trades), stats["Win Rate [%]"]]
            print(risk_metrics_list)
            df_risk.loc[len(df_risk)] = risk_metrics_list
        
        except:
            print(table, len(trades))
            pass
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_abnormal_day_ml", db_connection, if_exists="replace")

In [121]:
run_backtesting_ml(connection)

<ipython-input-119-667e6940dc82>:24: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_AR_long_short, cash=100_000, commission=.001)


['ADA_1min_complete_1day_preprocessed_1day_features_ensemble_pooling', 365, -9.140091666879743, -4.711656441717781, -9.140091666879743, -1.5087047485965515, -0.1881829587905829, 0.18109622229916061, -0.5047091292595177, 33, 48.484848484848484]
['ADA_1min_complete_1day_preprocessed_1day_features_knn_pooling', 365, -10.53427838543817, -4.711656441717781, -10.53427838543817, -1.7626904962639995, -0.21869522421120988, 0.6053674474115973, -0.17401461592426487, 178, 46.06741573033708]
['ADA_1min_complete_1day_preprocessed_1day_features_logistic_regression_pooling', 365, -69.59272716863, -4.711656441717781, -69.59272716863, -9.808425396107221, -1.4771525169257422, 0.7493752371965029, -0.9286766324036038, 92, 52.17391304347826]
ADA_1min_complete_1day_preprocessed_1day_features_mlp_classifier_pooling 0
['ADA_1min_complete_1day_preprocessed_1day_features_random_forest_pooling', 365, 35.27986550742056, -4.711656441717781, 35.27986550742056, 4.660161701480505, 0.7405741171933377, 0.419216752705467